In [ ]:
# Initialize drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Move to Google Drive 
%cd drive
%cd 'My Drive'
%cd 'MSc Stats Dissertation'

/content/drive
/content/drive/My Drive
/content/drive/My Drive/MSc Stats Dissertation


In [ ]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras as keras
import pandas as pd
import math
import tensorflow.keras.layers as layers
import deepsmiles
import rdkit
import selfies
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
BATCH_NORM = True
CONV_ACTIVATION = 'tanh'
CONV_DEPTH = 4
CONV_DIM_DEPTH = 32
CONV_DIM_WIDTH = 16
CONV_D_GF = 1.15875438383
CONV_W_GF = 1.1758149644
HIDDEN_DIM = 256

In [ ]:
def softmax_logits_loss_with_pad(labels,logits):
  weights = tf.cast(tf.not_equal(labels, 0), tf.float32)
  nonpad_seq = tf.math.count_nonzero(weights, dtype=tf.dtypes.float32, )
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits =logits)
  loss = loss *weights 
  return tf.reduce_sum(loss)

In [ ]:
## Neccesary CONSTANTS
BATCH_SIZE = 256
VOCAB_SIZE = len(vocab_index)
EPOCHS = 40
LEARNING_RATE =  0.000312087049936
DROP_OUT= 0.2
EMBEDDING_DIM = 192  ## Embedding dim of the characters
MAX_LEN = 101 ## Maximum size of a SMILE (100 + BOS, EOS)
PAD_LEN = 250 
MAX_LEN = PAD_LEN - 1
DROPOUT = 0.2
TRAIN = True

In [ ]:
## Import Necessary Data for training
train_smiles_X = np.load('./vocab/train_deep_smiles_X.npy',allow_pickle=True)
vocab =np.load('./vocab/deep_vocab.npy',allow_pickle=True)
vocab_index = np.load('./vocab/deep_vocab_index.npy',allow_pickle=True)
vocab = dict(vocab.ravel()[0])
vocab_index = dict(vocab_index.ravel()[0])

In [ ]:
index = np.where(train_smiles_X == 1)
t = np.split(train_smiles_X,index[0].tolist())
t= t[1:]
t = tf.keras.preprocessing.sequence.pad_sequences(t,maxlen=PAD_LEN,padding='post')

In [ ]:
NUM_BATCHES = math.floor(len(t)/BATCH_SIZE )

In [ ]:
NUM_TRAIN_BATCH = math.floor(NUM_BATCHES*0.99)
NUM_TEST_BATCH = math.floor(NUM_BATCHES*(0.01))

In [ ]:
test_X = t[NUM_TRAIN_BATCH*BATCH_SIZE:(NUM_TEST_BATCH+NUM_TRAIN_BATCH)*BATCH_SIZE]
train_X = t[:NUM_TRAIN_BATCH*BATCH_SIZE]

In [ ]:
## Cyclical linear annealing ##
def frange_cycle_linear(n_iter, start=0.0, stop=1.0,  n_cycle=4, ratio=0.5):
  L = np.ones(n_iter) * stop
  period = n_iter/n_cycle
  step = (stop-start)/(period*ratio) # linear schedule
  for c in range(n_cycle):
    v, i = start, 0
    while v <= stop and (int(i+c*period) < n_iter):
      L[int(i+c*period)] = v
      v += step
      i += 1
  return L 

In [ ]:
def train_smile_vae(vae,train_X, test_X,betas):
  clip = -1
  display_step = 100
  STEPS_PER_EPOCH = train_X.shape[0]//BATCH_SIZE
  TEST_STEPS = test_X.shape[0]//BATCH_SIZE
  LEARNING_RATE = 1e-4
  optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
  ## Index of current annealing 
  beta_ind = 0
  for epoch in range(EPOCHS):
    ### Ranomdize training data 
    indxs = np.arange(STEPS_PER_EPOCH)
    np.random.shuffle(indxs)
    
    for batch in range(STEPS_PER_EPOCH):
      with tf.GradientTape() as tape:
        ## Get relevant batch data
        X_batch = train_X[indxs[batch]*BATCH_SIZE: indxs[batch]*BATCH_SIZE + BATCH_SIZE ]
        ## Get logits of the model 
        z_mean, z_log_var, x_decoded = vae(X_batch[:,:-1])
        ## Get loss oppp
        loss_op = vae.vae_loss(labels=X_batch[:,1:],x_decoded =x_decoded,
                               z_mean =z_mean,z_log_var =z_log_var,beta=betas[beta_ind])
        
      ## Apply gradients
      gradients = tape.gradient(loss_op, vae.trainable_variables)
      if clip != -1:
        gradients, _ = tf.clip_by_global_norm(gradients, clip) 
      optimizer.apply_gradients(zip(gradients, vae.trainable_variables))

      ### Display information about training ###
      if batch % display_step == 0 or batch == 1:
        ### Get logits for test data
        rand_int = np.random.randint(low=0,high = TEST_STEPS)
        test_batch = test_X[rand_int*BATCH_SIZE: rand_int*BATCH_SIZE + BATCH_SIZE ]
        z_mean, z_log_var, x_decoded = vae(test_batch[:,:-1])
        test_loss = vae.vae_loss(labels=test_batch[:,1:],x_decoded =x_decoded,
                               z_mean =z_mean,z_log_var =z_log_var,beta= betas[beta_ind])
        
        ## Get KL Loss 
        kl_loss = vae.get_kl_loss(labels=test_batch[:,1:],x_decoded =x_decoded,
                               z_mean =z_mean,z_log_var =z_log_var,beta= betas[beta_ind])
        
        ## Get test logits  
        pred_test = tf.nn.softmax(x_decoded,axis =-1)
        weights = tf.cast(tf.not_equal(test_batch[:,1:], 0), tf.float32)
        nonpad_seq = tf.math.count_nonzero(weights, dtype=tf.dtypes.float32, )
        correct_pred_test = tf.equal(tf.argmax(pred_test,-1),test_batch[:,1:])

        ## Get test accuracy 
        accuracy_test = tf.reduce_sum(tf.cast(correct_pred_test, tf.float32))/nonpad_seq
        
        ### Print out test accuracy on model 
        print("Step " + str(batch) + ", Training Loss = " + \
                "{:.3f}".format(loss_op) + ", Test Loss = " + \
                "{:.3f}".format(test_loss)+ ", Test KL Loss = " + \
                "{:.3f}".format(kl_loss)+ ", Test Accuracy = " + \
                "{:.3f}".format(accuracy_test))
      beta_ind += 1
      ## Save every so often 
      if  (batch) % 3000 == 0:
        vae.save_weights('deep_conv_vae_weights2')

In [ ]:
betas = frange_cycle_linear(train_X.shape[0]//BATCH_SIZE *EPOCHS)
smile_vae = SMILE_VAE(vocab_size= VOCAB_SIZE,embedding_dim=EMBEDDING_DIM, max_len= MAX_LEN, 
                     latent_dim = 64, recurrent_dropout = DROP_OUT,dropout_rate= DROP_OUT)

In [ ]:
if TRAIN:
  train_smile_vae(smile_vae,train_X, test_X,betas)
else:
  smile_vae.load_weights('deep_conv_vae_weights2')

Step 0, Training Loss = 53415.719, Test Loss = 48626.492, Test KL Loss = 0.000, Test Accuracy = 0.142
Step 1, Training Loss = 55771.180, Test Loss = 46678.004, Test KL Loss = 0.000, Test Accuracy = 0.162
Step 100, Training Loss = 31131.412, Test Loss = 28179.459, Test KL Loss = 15.858, Test Accuracy = 0.352
Step 200, Training Loss = 29099.332, Test Loss = 26653.324, Test KL Loss = 24.567, Test Accuracy = 0.388
Step 300, Training Loss = 28035.578, Test Loss = 27322.838, Test KL Loss = 27.458, Test Accuracy = 0.375
Step 400, Training Loss = 25967.646, Test Loss = 26473.162, Test KL Loss = 26.963, Test Accuracy = 0.392
Step 500, Training Loss = 29904.375, Test Loss = 27187.695, Test KL Loss = 27.523, Test Accuracy = 0.379
Step 600, Training Loss = 26442.137, Test Loss = 25846.018, Test KL Loss = 24.544, Test Accuracy = 0.406
Step 700, Training Loss = 27691.420, Test Loss = 24998.852, Test KL Loss = 22.454, Test Accuracy = 0.424
Step 800, Training Loss = 24809.330, Test Loss = 24675.477, T